In [4]:
!pip install nibabel

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.1 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import pandas as pd
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2

os.getcwd()
os.chdir('/home/ubuntu/4catalyzer-hackathon/')

In [8]:
def convert_nifti_to_png_axial(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get a list of all files in the input directory
    file_list = os.listdir(input_dir)

    # Process each file in the input directory
    for file_name in file_list:
        file_path = os.path.join(input_dir, file_name)
        if os.path.isfile(file_path) and file_name.endswith(".nii.gz"):
            try:
                # Load the NIfTI file using nibabel
                nifti_image = nib.load(file_path)
                nifti_data = nifti_image.get_fdata()
                # Ensure the NIfTI data has 3 dimensions
                if len(nifti_data.shape) != 3:
                    raise ValueError(f"Invalid NIfTI file: {file_name} does not have 3 dimensions.")
                # Extract the filename without extension
                base_name = os.path.splitext(file_name)[0]
                # Save the axial slices as PNG images
                for i in range(nifti_data.shape[2]):
                    axial_slice = nifti_data[:, :, i]
                    png_file = os.path.join(output_dir, f"{base_name}_axial_slice_{i}.png")
                    cv2.imwrite(png_file, axial_slice)
                print(f"Converted {file_name} to PNG images (axial slices).")
            except Exception as e:
                print(f"Error converting {file_name}: {e}")

In [9]:
input_directory = "./data/ISLES-2022"
output_directory = "./data/lesion-images"
convert_nifti_to_png_axial(input_directory, output_directory)

In [15]:
def delete_files_without_flair(directory):
    # Get a list of all files in the directory
    file_list = os.listdir(directory)

    # Process each file in the directory
    for file_name in file_list:
        file_path = os.path.join(directory, file_name)
        # Check if the file name contains the word "FLAIR"
        if "FLAIR" not in file_name:
            try:
                # Delete the file
                os.remove(file_path)
                print(f"Deleted {file_name}.")
            except Exception as e:
                print(f"Error deleting {file_name}: {e}")

In [16]:
delete_files_without_flair(output_directory)

Deleted sub-strokecase0243_ses-0001_adc.nii_axial_slice_14.png.
Deleted sub-strokecase0138_ses-0001_adc.nii_axial_slice_13.png.
Deleted sub-strokecase0171_ses-0001_msk.nii_axial_slice_28.png.
Deleted sub-strokecase0013_ses-0001_dwi.nii_axial_slice_50.png.
Deleted sub-strokecase0088_ses-0001_msk.nii_axial_slice_42.png.
Deleted sub-strokecase0231_ses-0001_msk.nii_axial_slice_59.png.
Deleted sub-strokecase0050_ses-0001_dwi.nii_axial_slice_38.png.
Deleted sub-strokecase0085_ses-0001_dwi.nii_axial_slice_61.png.
Deleted sub-strokecase0080_ses-0001_adc.nii_axial_slice_20.png.
Deleted sub-strokecase0077_ses-0001_dwi.nii_axial_slice_71.png.
Deleted sub-strokecase0193_ses-0001_msk.nii_axial_slice_16.png.
Deleted sub-strokecase0101_ses-0001_msk.nii_axial_slice_0.png.
Deleted sub-strokecase0224_ses-0001_dwi.nii_axial_slice_54.png.
Deleted sub-strokecase0231_ses-0001_adc.nii_axial_slice_51.png.
Deleted sub-strokecase0020_ses-0001_adc.nii_axial_slice_4.png.
Deleted sub-strokecase0042_ses-0001_adc.ni

Deleted sub-strokecase0235_ses-0001_adc.nii_axial_slice_8.png.
Deleted sub-strokecase0010_ses-0001_msk.nii_axial_slice_44.png.
Deleted sub-strokecase0158_ses-0001_msk.nii_axial_slice_65.png.
Deleted sub-strokecase0174_ses-0001_msk.nii_axial_slice_28.png.
Deleted sub-strokecase0012_ses-0001_dwi.nii_axial_slice_39.png.
Deleted sub-strokecase0195_ses-0001_dwi.nii_axial_slice_10.png.
Deleted sub-strokecase0156_ses-0001_adc.nii_axial_slice_64.png.
Deleted sub-strokecase0145_ses-0001_adc.nii_axial_slice_19.png.
Deleted sub-strokecase0175_ses-0001_msk.nii_axial_slice_18.png.
Deleted sub-strokecase0136_ses-0001_adc.nii_axial_slice_62.png.
Deleted sub-strokecase0243_ses-0001_dwi.nii_axial_slice_9.png.
Deleted sub-strokecase0177_ses-0001_msk.nii_axial_slice_28.png.
Deleted sub-strokecase0032_ses-0001_dwi.nii_axial_slice_15.png.
Deleted sub-strokecase0057_ses-0001_adc.nii_axial_slice_67.png.
Deleted sub-strokecase0247_ses-0001_dwi.nii_axial_slice_32.png.
Deleted sub-strokecase0072_ses-0001_adc.ni

Deleted sub-strokecase0223_ses-0001_dwi.nii_axial_slice_12.png.
Deleted sub-strokecase0211_ses-0001_dwi.nii_axial_slice_2.png.
Deleted sub-strokecase0126_ses-0001_dwi.nii_axial_slice_61.png.
Deleted sub-strokecase0233_ses-0001_msk.nii_axial_slice_36.png.
Deleted sub-strokecase0145_ses-0001_msk.nii_axial_slice_40.png.
Deleted sub-strokecase0145_ses-0001_msk.nii_axial_slice_59.png.
Deleted sub-strokecase0108_ses-0001_msk.nii_axial_slice_59.png.
Deleted sub-strokecase0082_ses-0001_adc.nii_axial_slice_40.png.
Deleted sub-strokecase0087_ses-0001_dwi.nii_axial_slice_30.png.
Deleted sub-strokecase0159_ses-0001_dwi.nii_axial_slice_15.png.
Deleted sub-strokecase0172_ses-0001_msk.nii_axial_slice_56.png.
Deleted sub-strokecase0024_ses-0001_adc.nii_axial_slice_11.png.
Deleted sub-strokecase0088_ses-0001_dwi.nii_axial_slice_16.png.
Deleted sub-strokecase0001_ses-0001_msk.nii_axial_slice_14.png.
Deleted sub-strokecase0217_ses-0001_adc.nii_axial_slice_69.png.
Deleted sub-strokecase0201_ses-0001_adc.n

Deleted sub-strokecase0019_ses-0001_dwi.nii_axial_slice_70.png.
Deleted sub-strokecase0138_ses-0001_dwi.nii_axial_slice_45.png.
Deleted sub-strokecase0202_ses-0001_dwi.nii_axial_slice_0.png.
Deleted sub-strokecase0235_ses-0001_msk.nii_axial_slice_3.png.
Deleted sub-strokecase0022_ses-0001_msk.nii_axial_slice_70.png.
Deleted sub-strokecase0230_ses-0001_msk.nii_axial_slice_10.png.
Deleted sub-strokecase0142_ses-0001_dwi.nii_axial_slice_57.png.
Deleted sub-strokecase0087_ses-0001_adc.nii_axial_slice_10.png.
Deleted sub-strokecase0170_ses-0001_dwi.nii_axial_slice_23.png.
Deleted sub-strokecase0141_ses-0001_msk.nii_axial_slice_41.png.
Deleted sub-strokecase0214_ses-0001_dwi.nii_axial_slice_25.png.
Deleted sub-strokecase0135_ses-0001_msk.nii_axial_slice_9.png.
Deleted sub-strokecase0043_ses-0001_msk.nii_axial_slice_58.png.
Deleted sub-strokecase0141_ses-0001_adc.nii_axial_slice_49.png.
Deleted sub-strokecase0139_ses-0001_msk.nii_axial_slice_6.png.
Deleted sub-strokecase0172_ses-0001_msk.nii_

Deleted sub-strokecase0123_ses-0001_msk.nii_axial_slice_51.png.
Deleted sub-strokecase0107_ses-0001_dwi.nii_axial_slice_38.png.
Deleted sub-strokecase0034_ses-0001_dwi.nii_axial_slice_15.png.
Deleted sub-strokecase0160_ses-0001_dwi.nii_axial_slice_44.png.
Deleted sub-strokecase0234_ses-0001_dwi.nii_axial_slice_5.png.
Deleted sub-strokecase0015_ses-0001_adc.nii_axial_slice_10.png.
Deleted sub-strokecase0233_ses-0001_msk.nii_axial_slice_35.png.
Deleted sub-strokecase0242_ses-0001_dwi.nii_axial_slice_34.png.
Deleted sub-strokecase0163_ses-0001_dwi.nii_axial_slice_60.png.
Deleted sub-strokecase0007_ses-0001_msk.nii_axial_slice_26.png.
Deleted sub-strokecase0152_ses-0001_dwi.nii_axial_slice_37.png.
Deleted sub-strokecase0046_ses-0001_dwi.nii_axial_slice_50.png.
Deleted sub-strokecase0227_ses-0001_msk.nii_axial_slice_14.png.
Deleted sub-strokecase0052_ses-0001_dwi.nii_axial_slice_23.png.
Deleted sub-strokecase0064_ses-0001_msk.nii_axial_slice_62.png.
Deleted sub-strokecase0092_ses-0001_msk.n

...

Deleted sub-strokecase0022_ses-0001_adc.nii_axial_slice_60.png.
Deleted sub-strokecase0020_ses-0001_adc.nii_axial_slice_32.png.
Deleted sub-strokecase0070_ses-0001_dwi.nii_axial_slice_49.png.
Deleted sub-strokecase0242_ses-0001_dwi.nii_axial_slice_12.png.
Deleted sub-strokecase0170_ses-0001_dwi.nii_axial_slice_28.png.
Deleted sub-strokecase0120_ses-0001_msk.nii_axial_slice_13.png.
Deleted sub-strokecase0178_ses-0001_dwi.nii_axial_slice_54.png.
Deleted sub-strokecase0217_ses-0001_dwi.nii_axial_slice_63.png.
Deleted sub-strokecase0066_ses-0001_dwi.nii_axial_slice_16.png.
Deleted sub-strokecase0226_ses-0001_adc.nii_axial_slice_12.png.
Deleted sub-strokecase0217_ses-0001_adc.nii_axial_slice_22.png.
Deleted sub-strokecase0082_ses-0001_msk.nii_axial_slice_17.png.
Deleted sub-strokecase0156_ses-0001_dwi.nii_axial_slice_44.png.
Deleted sub-strokecase0229_ses-0001_msk.nii_axial_slice_16.png.
Deleted sub-strokecase0043_ses-0001_dwi.nii_axial_slice_19.png.
Deleted sub-strokecase0156_ses-0001_msk.

Deleted sub-strokecase0039_ses-0001_dwi.nii_axial_slice_46.png.
Deleted sub-strokecase0238_ses-0001_msk.nii_axial_slice_25.png.
Deleted sub-strokecase0061_ses-0001_adc.nii_axial_slice_17.png.
Deleted sub-strokecase0109_ses-0001_msk.nii_axial_slice_27.png.
Deleted sub-strokecase0048_ses-0001_adc.nii_axial_slice_11.png.
Deleted sub-strokecase0057_ses-0001_msk.nii_axial_slice_0.png.
Deleted sub-strokecase0042_ses-0001_dwi.nii_axial_slice_16.png.
Deleted sub-strokecase0001_ses-0001_msk.nii_axial_slice_5.png.
Deleted sub-strokecase0042_ses-0001_msk.nii_axial_slice_27.png.
Deleted sub-strokecase0214_ses-0001_adc.nii_axial_slice_16.png.
Deleted sub-strokecase0202_ses-0001_dwi.nii_axial_slice_20.png.
Deleted sub-strokecase0033_ses-0001_msk.nii_axial_slice_21.png.
Deleted sub-strokecase0013_ses-0001_msk.nii_axial_slice_38.png.
Deleted sub-strokecase0236_ses-0001_adc.nii_axial_slice_21.png.
Deleted sub-strokecase0225_ses-0001_dwi.nii_axial_slice_59.png.
Deleted sub-strokecase0179_ses-0001_msk.ni

Deleted sub-strokecase0109_ses-0001_msk.nii_axial_slice_3.png.
Deleted sub-strokecase0108_ses-0001_msk.nii_axial_slice_44.png.
Deleted sub-strokecase0129_ses-0001_msk.nii_axial_slice_23.png.
Deleted sub-strokecase0065_ses-0001_adc.nii_axial_slice_34.png.
Deleted sub-strokecase0161_ses-0001_adc.nii_axial_slice_22.png.
Deleted sub-strokecase0154_ses-0001_adc.nii_axial_slice_25.png.
Deleted sub-strokecase0196_ses-0001_adc.nii_axial_slice_1.png.
Deleted sub-strokecase0212_ses-0001_msk.nii_axial_slice_17.png.
Deleted sub-strokecase0095_ses-0001_msk.nii_axial_slice_46.png.
Deleted sub-strokecase0085_ses-0001_dwi.nii_axial_slice_35.png.
Deleted sub-strokecase0065_ses-0001_adc.nii_axial_slice_36.png.
Deleted sub-strokecase0102_ses-0001_dwi.nii_axial_slice_1.png.
Deleted sub-strokecase0062_ses-0001_dwi.nii_axial_slice_31.png.
Deleted sub-strokecase0193_ses-0001_msk.nii_axial_slice_21.png.
Deleted sub-strokecase0244_ses-0001_msk.nii_axial_slice_22.png.
Deleted sub-strokecase0163_ses-0001_adc.nii

Deleted sub-strokecase0132_ses-0001_adc.nii_axial_slice_59.png.
Deleted sub-strokecase0083_ses-0001_adc.nii_axial_slice_43.png.
Deleted sub-strokecase0122_ses-0001_adc.nii_axial_slice_46.png.
Deleted sub-strokecase0107_ses-0001_dwi.nii_axial_slice_46.png.
Deleted sub-strokecase0143_ses-0001_msk.nii_axial_slice_72.png.
Deleted sub-strokecase0082_ses-0001_msk.nii_axial_slice_57.png.
Deleted sub-strokecase0035_ses-0001_msk.nii_axial_slice_47.png.
Deleted sub-strokecase0029_ses-0001_msk.nii_axial_slice_9.png.
Deleted sub-strokecase0069_ses-0001_msk.nii_axial_slice_28.png.
Deleted sub-strokecase0053_ses-0001_adc.nii_axial_slice_56.png.
Deleted sub-strokecase0211_ses-0001_dwi.nii_axial_slice_19.png.
Deleted sub-strokecase0160_ses-0001_dwi.nii_axial_slice_10.png.
Deleted sub-strokecase0039_ses-0001_adc.nii_axial_slice_34.png.
Deleted sub-strokecase0163_ses-0001_dwi.nii_axial_slice_1.png.
Deleted sub-strokecase0071_ses-0001_msk.nii_axial_slice_59.png.
Deleted sub-strokecase0061_ses-0001_adc.ni

Deleted sub-strokecase0148_ses-0001_adc.nii_axial_slice_46.png.
Deleted sub-strokecase0134_ses-0001_adc.nii_axial_slice_72.png.
Deleted sub-strokecase0016_ses-0001_adc.nii_axial_slice_50.png.
Deleted sub-strokecase0196_ses-0001_adc.nii_axial_slice_57.png.
Deleted sub-strokecase0087_ses-0001_adc.nii_axial_slice_27.png.
Deleted sub-strokecase0234_ses-0001_msk.nii_axial_slice_26.png.
Deleted sub-strokecase0203_ses-0001_adc.nii_axial_slice_46.png.
Deleted sub-strokecase0133_ses-0001_msk.nii_axial_slice_41.png.
Deleted sub-strokecase0008_ses-0001_msk.nii_axial_slice_4.png.
Deleted sub-strokecase0130_ses-0001_msk.nii_axial_slice_26.png.
Deleted sub-strokecase0135_ses-0001_msk.nii_axial_slice_33.png.
Deleted sub-strokecase0086_ses-0001_dwi.nii_axial_slice_6.png.
Deleted sub-strokecase0234_ses-0001_msk.nii_axial_slice_47.png.
Deleted sub-strokecase0039_ses-0001_dwi.nii_axial_slice_47.png.
Deleted sub-strokecase0142_ses-0001_msk.nii_axial_slice_48.png.
Deleted sub-strokecase0248_ses-0001_adc.ni